In [ ]:
#  Spot hkl recognition by neighbourhood distances distribution

In [1]:
#%matplotlib inline
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.__version__
import time,copy,os
import LaueTools.dict_LaueTools as dictLT
import LaueTools.findorient as FO
import LaueTools.generaltools as GT
import LaueTools.IOLaueTools as IOLT
import LaueTools.lauecore as LT
import LaueTools.LaueGeometry as LTGeo
from tqdm import trange
# third party LaueTools import
import LaueTools.CrystalParameters as CP
import LaueTools.readmccd as RMCCD

-- OK! You are using python 3
Missing fabio module. Please install it if you need open some tiff images from the sCMOS camera
Missing library libtiff, Please install: pylibtiff if you need open some tiff images


In [2]:
# primary functions for cubic materials
def classhkl_cubic(hkl):
    """ unique representation of hkl for cubic case
    """
    return sorted([abs(int(ind)) for ind in hkl])

def monoclinic_sameclass(hkl1, hkl2):
    """
    hkl ≡ -hk-l ≡ -h-k-l ≡ h-kl
    """
    h,k,l = hkl1
    h1,k1,l1 = hkl2
    if (h==h1):
        if (k==k1) or (k==-k1):
            if (l==l1):
                return True
    elif (h==-h1):
        if (k==k1) or (k==-k1):
            if (l==-l1):
                return True
    
def areparallel(a,b):
    """test if vectors a b colinear (same direction)"""
    return np.sum(np.cross(a,b)**2)==0.

def commonclass(hkl1, hkl2, material_):
    """ test if hkl1 and hkl2 belong to the same class"""
    if material_ == "Cu":
        cond1 = classhkl_cubic(hkl1)==classhkl_cubic(hkl2)
        cond2 = areparallel(classhkl_cubic(hkl1),classhkl_cubic(hkl2))
    elif material_ == "ZrO2":
        cond1 = monoclinic_sameclass(hkl1, hkl2)
        cond2 = areparallel(classhkl_cubic(hkl1),classhkl_cubic(hkl2))
    return cond1 or cond2

def getclassindex(hkl, classhkl, material_):
    """
    return index in hklclasses of reprensative for given hkl
    """
    if material_ == "ZrO2":
        hkl_c = hkl
    elif material_ == "Cu":
        hkl_c = classhkl_cubic(hkl)
    diff = np.sum((hkl_c - classhkl)**2,axis=1)
    posindex = np.where(diff==0)[0]
    assert len(posindex)<=1
    return posindex

# Funtion to get the material data and fingerprint of the hkls
def get_material_data(material_="Cu", ang_maxx = 45, step = 0.5, hkl_ref=13, hklclass=11,verbose=0):
    ## material parameters
    a, b, c, alpha, beta, gamma = dictLT.dict_Materials[material_][1]
    Gstar = CP.Gstar_from_directlatticeparams(a, b, c, alpha, beta, gamma)
    rules = dictLT.dict_Materials[material_][-1]
    ## building a reference hkl list
    hkl2 = GT.threeindices_up_to(hkl_ref)
    hkl2 = CP.ApplyExtinctionrules(hkl2,rules)
    ## build hklclass
    n = hklclass
    classhkl = []
    if material_ == "Cu":
        for i in range(1,n+1):
            for j in range(0,i+1):
                for k in range(0,j+1):
                    classhkl.append([i,j,k])
        classhkl = np.array(classhkl)
        classhkl = CP.ApplyExtinctionrules(classhkl, rules)
        classhkl = CP.FilterHarmonics_2(classhkl)  
        classhkl = np.array([classhkl_cubic(hkl) for hkl in classhkl])
        
    elif material_ == "ZrO2":
        for i1 in range(0,n+1):
            for j1 in range(0,n+1):
                for k1 in range(0,n+1):
                    comb = permutations([i1,j1,k1], 3)
                    for i in comb:
                        h,k,l = i
                        classhkl.append([h,k,l])
                        classhkl.append([-h,-k,l])
                        classhkl.append([h,-k,-l])
                        classhkl.append([-h,k,l])
                        classhkl.append([h,k,-l])
        classhkl = np.array(classhkl)
        classhkl = np.unique(classhkl, axis=0)
        classhkl = CP.ApplyExtinctionrules(classhkl, rules)
        classhkl = CP.FilterHarmonics_2(classhkl)
    #%
    codebars = []
    fingerprint_all = {}
    for kl, hkl in enumerate(classhkl):
        hkl1 = np.array(hkl)
        # get angular distance of given hkl to the hkl list
        (pp, angles), LUT = FO.PlanePairs_from2sets(ang_maxx/2., ang_maxx/2.,
                                                    [hkl1,], hkl2, material_, 
                                                    onlyclosest=0, verbose=verbose)
        angbins = np.arange(0,ang_maxx+step,step) # linear binning ftm
        fingerprint = np.histogram(angles, bins=angbins, density=False)[0]
        binnedangles = np.round(angles, decimals=5)
        ind = np.digitize(angles, angbins) - 1  # subtract 1 for edge bin
        relevant_hkl = pp[:,1,:]
        
        ## generate a sub list of relevant HKL planes in the binned fingerprint
        ## location of reference hkl in the bins
        temp_loc = np.zeros(len(relevant_hkl), dtype=np.int8)
        if material_ == "Cu":
            for k, i in enumerate(classhkl):
                for l, j in enumerate(relevant_hkl):
                    isok = commonclass(i,j,material_)
                    if isok:
                        temp_loc[l] = k
                        
        fingerprint_all[kl] = {'fingerprint': fingerprint,
                                'ind' : ind,
                                'relevant_hkl' : relevant_hkl,
                                'equivalent_hkl' : temp_loc,
                                'classhkl' : hkl}
        
        codebars.append(fingerprint)
        
    assert len(codebars) == len(classhkl)
    
    return codebars, fingerprint_all, angbins, classhkl, hkl2

In [3]:
material_ = 'Cu' #'Cu' 
codebars, fingerprint_all, angbins, classhkl, hkl2 = get_material_data(material_=material_,\
                                                                        ang_maxx = 25, \
                                                                        step = 0.1, \
                                                                        hkl_ref=9, \
                                                                        hklclass=9,
                                                                        verbose =0)

Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from2sets()
Calculating LUT in PlanePairs_from

In [4]:
def simulatemultiplepatterns(n,seed=1234,key_material=None,emax=23,
                             detectorparameters=None,
                            sortintensity = False):
    nbUBs=n
    np.random.seed(seed)
    UBelemagnles = np.random.random((3,nbUBs))*360-180.
    l_tth, l_chi, l_miller_ind, l_posx, l_posy, l_E, l_intensity = [],[],[],[],[],[],[]
    
    for angle_X, angle_Y, angle_Z in UBelemagnles.T:
        UBmatrix = GT.fromelemangles_toMatrix([angle_X, angle_Y, angle_Z])
        grain = CP.Prepare_Grain(key_material, UBmatrix)
        # a bit distorsion
        #Grain[2] = np.dot(Grain[2], self.Bmatrix)
        if detectorparameters is None:
            detectorparameters = [77.088, 1012.45, 1049.92, 0.423, 0.172]  # sCMOS
            detectorparameters = [70, 1024, 1024, 0, 0]
            detectorparameters = [70.182, 1039.309, 944.122, 0.747, 0.071]
        s_tth, s_chi, s_miller_ind, s_posx, s_posy, s_E= LT.SimulateLaue_full_np(grain, 5, 23,
                                                                                 detectorparameters,
                                                                                 pixelsize=0.079142,
                                                                                 removeharmonics=1)
        s_intensity = 1./s_E
        l_tth.append(s_tth)
        l_chi.append(s_chi)
        l_miller_ind.append(s_miller_ind)
        l_posx.append(s_posx)
        l_posy.append(s_posy)
        l_E.append(s_E)
        l_intensity.append(s_intensity)
    #flat_list = [item for sublist in l for item in sublist]
    s_tth = np.array([item for sublist in l_tth for item in sublist])
    s_chi = np.array([item for sublist in l_chi for item in sublist])
    s_miller_ind = np.array([item for sublist in l_miller_ind for item in sublist])
    s_posx = np.array([item for sublist in l_posx for item in sublist])
    s_posy = np.array([item for sublist in l_posy for item in sublist])
    s_E = np.array([item for sublist in l_E for item in sublist])
    s_intensity=np.array([item for sublist in l_intensity for item in sublist])
    
    if sortintensity:
        indsort = np.argsort(s_intensity)[::-1]
        s_tth=np.take(s_tth, indsort)
        s_chi=np.take(s_chi, indsort)
        s_miller_ind=np.take(s_miller_ind, indsort, axis=0)
        s_posx=np.take(s_posx, indsort)
        s_posy=np.take(s_posy, indsort)
        s_E=np.take(s_E, indsort)
        s_intensity=np.take(s_intensity, indsort)
    return s_tth, s_chi, s_miller_ind,s_posx,s_posy

def spots_in_ROI(s_tth,s_chi,centerscattering=[70/2.,0],anglefromcenter=10.):
    """ return spot indices of spot lying in a given direction around a central direction"""
    # finding central spots
    s_the = s_tth/2.
    exp_s = np.array([s_the,s_chi]).T
    dist=GT.calculdist_from_thetachi(np.array([centerscattering]),exp_s).flatten()
    conddistin = dist<=anglefromcenter
    condtth = s_tth<70
    cond = np.logical_and(conddistin,condtth)
    #cond = np.logical_and(dist>anglefromcenter,condtth)
    spots_ix= np.where(cond)[0]
    return spots_ix

def prepare_LP(nbgrains, seednumber, material_,verbose,plotLauePattern):
    s_tth, s_chi, s_miller_ind, s_posx,s_posy = simulatemultiplepatterns(nbgrains,
                                                                        seed=seednumber,
                                                                        key_material=material_,
                                                                        sortintensity=True)
    spots_in_center = spots_in_ROI(s_tth,s_chi)
    if verbose:
        print('nb of central spots',len(spots_in_center))
        print('Central spots',spots_in_center)
    
    if plotLauePattern:
        figlaue, axlaue= plt.subplots()
        axlaue.scatter(s_tth,s_chi)
        axlaue.scatter(s_tth[spots_in_center],s_chi[spots_in_center], c="r")
        axlaue.set_title('nb matrices: %d, nb spots : %d'%(nbgrains,len(s_tth)))
    # considering all spots
    allspots_the_chi = np.transpose(np.array([s_tth/2., s_chi]))
    #print("Calculating all mutual angular distances of selected spots...")
    tabledistancerandom = np.transpose(GT.calculdist_from_thetachi(allspots_the_chi, allspots_the_chi))
    # ground truth
    hkl_sol = s_miller_ind
    return tabledistancerandom, hkl_sol, spots_in_center, s_tth

In [7]:
from scipy.spatial.distance import cityblock, braycurtis,\
    canberra,chebyshev,correlation,cosine,euclidean,jensenshannon,\
        minkowski,sqeuclidean

def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

def findhkl(spotindex, tabledistance, cl_codebars, codebar_angbins, classhkl, hkl_sol=None,
            threshold_diff = 0.25,
            plotresults=False,
            verbose=0,
            method='dot',
            nbpredictions = 1,
            cl_fingerprint= None,
            material_=None,
           reduced_data=0):
    """
    predict hkl of a given spot by finding similar local radial distances histogram
    cl_codebars :  list of class features
    classhkl :  list of class hkl
    """
    spotangles = tabledistance[spotindex]
    spotangles = np.delete(spotangles, spotindex)# removing the self distance
    ind = np.digitize(spotangles, codebar_angbins) - 1 ## keep track of indices
    cb_exp = np.histogram(spotangles, bins=codebar_angbins)[0]
    
    if hkl_sol is not None:
        class_sol = hkl_sol[spotindex]
    else:
        print('Missing "hkl_sol" , target of miller indices')
        return

    nbclasses = len(classhkl)
    corresults = np.zeros(nbclasses)
    corresults_sub = np.zeros(nbclasses)
    corresults_ = {}
    diff10 = 0
    # not enough information ! exp. spot fingerprint is zero everywhere
    # or number of markers is too low
    if np.dot(cb_exp,cb_exp)== 0. or len(np.where(cb_exp>0.)[0])<=2:
        isgoodprediction = 'Not feasible'
        bestindex = -1
        class_predicted = None
        predicted_class_index = -1
        return class_predicted, predicted_class_index, corresults, isgoodprediction
    else:
        intersect_list = []
        intersect_list_theo = []
        intersect_list_sub = []
        for cl_i in range(nbclasses):
            ## Experimental pattern
            cb_exp = np.histogram(spotangles, bins=codebar_angbins)[0]
            ## Theoretical pattern
            cb_theo = cl_codebars[cl_i]
            
            ## Simple way of findign intersecting bins
            theo_ind = np.where(cb_theo > 0)[0]
            exp_ind = np.where(cb_exp > 0)[0]
            count_intersect = np.intersect1d(exp_ind, theo_ind)
            
            intersect_list.append(len(count_intersect))
            intersect_list_theo.append(len(theo_ind))
            
            if reduced_data:
                for ijk in range(len(cb_exp)):
                    if cb_theo[ijk] == 0:
                        cb_exp[ijk] = 0

                if len(count_intersect) > 0:
                    # cumcb1 = np.cumsum(cb_theo[count_intersect])
                    # cumcb2 = np.cumsum(cb_exp[count_intersect])
                    cumcb1 = np.cumsum(cb_theo)
                    cumcb2 = np.cumsum(cb_exp)
                    cb_theo = cumcb1/cumcb1[-1]
                    cb_exp = cumcb2/cumcb2[-1]

                else:
                    cumcb1 = np.cumsum(cb_theo)
                    cumcb2 = np.cumsum(cb_exp)
                    cb_theo = cumcb1/cumcb1[-1]
             ## calculate the pearson correlation between histograms
            global_score = np.round(np.corrcoef(cb_exp, cb_theo)[0,1],3)
            corresults[cl_i] = global_score           
                
            ## calculate different types of distances between histograms
            ## not all are useful, some may cause overfitting
            ## Euclidean distances for discrete histograms are bad!!!
            d1 = cityblock(cb_exp, cb_theo)
            d2 = braycurtis(cb_exp, cb_theo)
            d3 = canberra(cb_exp, cb_theo) # 
            d4 = chebyshev(cb_exp, cb_theo) # 
            d5 = correlation(cb_exp, cb_theo)
            d6 = cosine(cb_exp, cb_theo) ## values between 1 to -1
            d7 = euclidean(cb_exp, cb_theo)
            d8 = jensenshannon(cb_exp, cb_theo)
            d9 = minkowski(cb_exp, cb_theo)
            d10 = sqeuclidean(cb_exp, cb_theo)
            d11 = 0.5 * np.sum([((a - b) ** 2) / (a + b + 1e-10) for (a, b) in zip(cb_exp, cb_theo)])
            d12 = jaccard_similarity(cb_exp, cb_theo)
            corresults_[cl_i] = {'d1':d1,'d2':d2,'d3':d3,'d4':d4,'d5':d5,
                                  'd6':d6,'d7':d7,'d8':d8,'d9':d9,'d10':d10,'d11':d11,'d12':d12}
            
            ### generating scores based on the fitting of bins
            ## so far no good results: to comeback later
            count = 0
            ## calculate the sub score based on sort of tree FP
            sub_score = 0
            sub_inter = 0
            if len(count_intersect) > 0:
                #sub_stats = {}
                for i in count_intersect:
                    # if i > 100:
                    #     # skipping higher angular deviation bins
                    #     continue
                    int1 = np.where(cl_fingerprint[cl_i]['ind']==i)[0] # location of intersecting equivalent HKL from FP library
                    int1exp = np.where(ind==i)[0]
                    # Loop through all possible intersection HKL of a bin
                    for k in int1:
                        sub_fingerprint_ind = cl_fingerprint[cl_i]['equivalent_hkl'][k]
                        for kexp in int1exp:
                            spotangles_new = tabledistance[kexp]
                            spotangles_new = np.delete(spotangles_new, kexp)# removing the self distance
                            cb_exp_new = np.histogram(spotangles_new, bins=codebar_angbins)[0]
                            cb_theo_new = cl_codebars[sub_fingerprint_ind]
                            ## calcualte simple intersection of bins
                            theo_ind1 = np.where(cb_theo_new > 0)[0]
                            exp_ind1 = np.where(cb_exp_new > 0)[0]
                            count_intersect1 = np.intersect1d(exp_ind1, theo_ind1)
                            sub_inter = sub_inter + len(count_intersect1)/len(theo_ind1)
                            ## calculate pearson correlation for sub bins
                            sub_score = sub_score + np.corrcoef(cb_theo_new, cb_exp_new)[0,1]
                            
                            # pearson_score = np.corrcoef(cb_theo_new, cb_exp_new)[0,1]
                            #sub_stats[str(i)] ={"theo_HKL": classhkl[sub_fingerprint_ind],
                             #                      "Exp_index": kexp,
                              #                     "intersect_score": len(count_intersect1)
                               #                    "theoretical_score":len(theo_ind1),
                                #                   "correlation_score":pearson_score}
                            count = count + 1
                            
            intersect_list_sub.append(len(count_intersect) + sub_inter)
            if count == 0:
                corresults_sub[cl_i] = global_score + (sub_score)
            else:
                corresults_sub[cl_i] = global_score + (sub_score/ count)
# =============================================================================
        # checking if the results is outstanding
        sind = np.argsort(corresults)[::-1]
        diff10 = corresults[sind[0]]-corresults[sind[1]]
        
        intersect_list_theo = np.array(intersect_list_theo)
        intersect_list = np.array(intersect_list)
        sind_il = np.argsort(intersect_list)[::-1]

        get_max_elems = len(np.unique(intersect_list))
        if get_max_elems > 6: # limiting to top 3 list
            get_max_elems = 6

        first_twomax = np.sort(np.unique(intersect_list))[-get_max_elems:][::-1]    
        if  abs(np.diff(first_twomax[:2])) < 10 and  abs(np.diff(first_twomax[:2])) > 0:
            temp_list = []
            for i in range(get_max_elems):
                temp_list0 = np.where(intersect_list == first_twomax[i])[0]
                temp_list = temp_list + temp_list0.flatten().tolist()
        else:
            temp_list = np.argwhere(intersect_list == np.amax(intersect_list))
            temp_list = temp_list.flatten().tolist()

        temp_list = np.sort(temp_list)

        if len(temp_list) == 1: # clear choice baesd on bin intersection (more robust)
            bestindex = temp_list[0]
            class_predicted = classhkl[bestindex]
            isgoodprediction = commonclass(class_predicted,class_sol,material_)
            predicted_class_index = getclassindex(class_predicted,classhkl,material_)
            
        elif (diff10 > threshold_diff): # check if threshold is satisfied
            bestindex= np.argmax(corresults)
            class_predicted = classhkl[bestindex]
            isgoodprediction = commonclass(class_predicted,class_sol,material_)
            predicted_class_index = getclassindex(class_predicted,classhkl,material_)
        
        elif intersect_list[sind_il[0]] == intersect_list[sind_il[1]]: # two HKLs show same length of intersection, compare with theo intersections to compute ratios    
            r1 = intersect_list[sind_il[0]] / intersect_list_theo[sind_il[0]]
            r2 = intersect_list[sind_il[1]] / intersect_list_theo[sind_il[1]]
            r3 = intersect_list[sind_il[2]] / intersect_list_theo[sind_il[2]]
            r4 = intersect_list[sind_il[3]] / intersect_list_theo[sind_il[3]]
            r5 = intersect_list[sind_il[4]] / intersect_list_theo[sind_il[4]]
            r6 = intersect_list[sind_il[5]] / intersect_list_theo[sind_il[5]]
            if verbose:
                print(r1,r2,r3,r4,r5,r6)
            ind_best = [r1,r2,r3,r4,r5,r6]
            ind7 = np.argmax(ind_best)
            
            bestindex = sind_il[ind7]
            class_predicted = classhkl[bestindex]
            isgoodprediction = commonclass(class_predicted,class_sol,material_)
            predicted_class_index = getclassindex(class_predicted,classhkl,material_)

        else: # else compare the distances to suggest a better choice among the possible HKLs
            minimum_ind = []
            for ij in range(1,13):
                if ij in [6,7,10]: # skipping some distances (may produce wrong results)
                    continue
                # if ij not in [9,12]: # skipping some distances (may produce wrong results)
                #     continue
                temp_ = [corresults_[i]['d'+str(ij)] for i in range(len(corresults_)) if i in temp_list]
                temp_ = np.array(temp_)
                minimum_ind.append(np.argmin(temp_))
                
            if verbose:
                print(classhkl[temp_list])
                print(minimum_ind)

            if len(np.unique(minimum_ind)) > 4: # more than two HKLs show minimum distances
                ## No concrete results from distance analysis;
                ## Take results of best pearson correlation
                bestindex= np.argmax(corresults)
                class_predicted = classhkl[bestindex]
                isgoodprediction = commonclass(class_predicted,class_sol,material_)
                predicted_class_index = getclassindex(class_predicted,classhkl,material_)
            else:
                import collections
                counter=collections.Counter(minimum_ind)
                trial_ = counter.most_common(1)[0]

                if trial_[1] > 3: # distance count should be above 3 times for a given HKL 
                    tempi_ = [intersect_list[itemp] / intersect_list_theo[itemp] for itemp in temp_list]
                    arg_max = np.argmax(tempi_)
                    if arg_max == trial_[0]:
                        bestindex = temp_list[trial_[0]]
                        class_predicted = classhkl[bestindex]
                        isgoodprediction = commonclass(class_predicted,class_sol,material_)
                        predicted_class_index = getclassindex(class_predicted,classhkl,material_)
                    else: # take the best correlation results from Pearson
                        if verbose:
                            print("Argmax of intersection ratio is used")
                            print(tempi_)
                        bestindex = temp_list[arg_max]
                        class_predicted = classhkl[bestindex]
                        isgoodprediction = commonclass(class_predicted,class_sol,material_)
                        predicted_class_index = getclassindex(class_predicted,classhkl,material_)

                else: # take the best correlation results from Pearson
                    #bestindex= np.argmax(corresults[temp_list])
                    #class_predicted = classhkl[temp_list[bestindex]]
                    bestindex= np.argmax(corresults)
                    class_predicted = classhkl[bestindex]
                    isgoodprediction = commonclass(class_predicted,class_sol,material_)
                    predicted_class_index = getclassindex(class_predicted,classhkl,material_)

    if verbose:
        print("**")
        print("Solution: ",class_sol, isgoodprediction)
        print("** Intersection list GLOBAL")
        sind = np.argsort(intersect_list)[::-1]
        print("First 4: ",classhkl[sind[0]],classhkl[sind[1]],classhkl[sind[2]],classhkl[sind[3]])
        print("First 4 Score: ",intersect_list[sind[0]],intersect_list[sind[1]],
              intersect_list[sind[2]],intersect_list[sind[3]])
        print("** Intersection list SUB")
        intersect_list_sub = np.array(intersect_list_sub)
        sind = np.argsort(intersect_list_sub)[::-1]
        print("First 4: ",classhkl[sind[0]],classhkl[sind[1]],classhkl[sind[2]],classhkl[sind[3]])
        print("First 4 Score: ",intersect_list_sub[sind[0]],intersect_list_sub[sind[1]],
              intersect_list_sub[sind[2]],intersect_list_sub[sind[3]])
        print("** Correl list GLOBAL")
        sind = np.argsort(corresults)[::-1]
        print("First 4: ",classhkl[sind[0]],classhkl[sind[1]],classhkl[sind[2]],classhkl[sind[3]])
        print("First 4 Score: ",corresults[sind[0]],corresults[sind[1]],
              corresults[sind[2]],corresults[sind[3]])
        print("** Correl list SUB")
        sind = np.argsort(corresults_sub)[::-1]
        print("First 4: ",classhkl[sind[0]],classhkl[sind[1]],classhkl[sind[2]],classhkl[sind[3]])
        print("First 4 Score: ",corresults_sub[sind[0]],corresults_sub[sind[1]],
              corresults_sub[sind[2]],corresults_sub[sind[3]])        
        print("***************************************************************")
        # class_label = hkls[best_class_index]
        # print('matching rates:',corresults)
        print('predicted_class_index', predicted_class_index)
        print("class_sol",class_sol)
        print('Spot %d . Predicted Class: %s'%(spotindex, str(class_predicted)))
        print('diff10: ',diff10)
        
        
    if not isgoodprediction:
        sind = np.argsort(intersect_list)[::-1]
        sind1 = np.argsort(intersect_list_sub)[::-1]
        sind2 = np.argsort(corresults)[::-1]
        sind3 = np.argsort(corresults_sub)[::-1]
        tt_ = [classhkl[sind[0]],classhkl[sind[1]],classhkl[sind[2]],classhkl[sind[3]],
               classhkl[sind1[0]],classhkl[sind1[1]],classhkl[sind1[2]],classhkl[sind1[3]],
               classhkl[sind2[0]],classhkl[sind2[1]],classhkl[sind2[2]],classhkl[sind2[3]],
               classhkl[sind3[0]],classhkl[sind3[1]],classhkl[sind3[2]],classhkl[sind3[3]]]
        if material_ == "Cu":
            class_sol = classhkl_cubic(class_sol)
        if class_sol in np.array(tt_):
            isgoodprediction = "possible_in_best_4_values"
            
    return class_predicted, predicted_class_index, corresults, isgoodprediction

In [9]:
nb_goods = 0
nb_wrongs = 0
nb_neutral = 0
nbtestspots = 0
wrongpredictionList=[]
goodpredictionList=[]
verbose = 0
verbose1 = 1
threshold_diff10 = 0.12
nbgrains = 3

for _ in range(50):
    seednumber = np.random.randint(10000000)

    tabledistancerandom, hkl_sol, spots_in_center, s_tth = prepare_LP(nbgrains, 
                                                               seednumber, 
                                                               material_,
                                                               verbose,
                                                               plotLauePattern=False)
    # spotindices = range(10)
    spotindices = spots_in_center

    nbtestspots = nbtestspots + len(spotindices)
    
    for sp_nb, spotindex in enumerate(spotindices):
        # if sp_nb >= 1:
        #     continue
        predicted_class, classindex, matchvector, goodprediction = findhkl(spotindex, tabledistancerandom,
                                                                           codebars,
                                                                           angbins,
                                                                           classhkl, 
                                                                           hkl_sol=hkl_sol,
                                                                           threshold_diff=threshold_diff10,
                                                                           verbose=verbose, method='new',
                                                                           plotresults=False,
                                                                           cl_fingerprint=fingerprint_all,
                                                                           material_=material_)
        if verbose:
            print("seed :", seednumber, goodprediction)
            print('solution class',hkl_sol[spotindex])
            print('Predicted class',classhkl[classindex])
            print('goodprediction',goodprediction)
        if goodprediction == True:
            if verbose: print('***** GOOD PREDICTION ! *****')
            nb_goods += 1
            goodpredictionList.append([spotindex, predicted_class, hkl_sol[spotindex], matchvector])
        elif goodprediction == False:
            if verbose: print('WRONG PREDICTION :[ ')
            wrongpredictionList.append([spotindex, predicted_class, hkl_sol[spotindex], matchvector])
            nb_wrongs += 1
        else:
            if verbose: print('goodprediction', goodprediction)
            nb_neutral += 1
            pass

if verbose1:
    print("***********")        
    print('Total Prediction: ',nbtestspots) 
    print('GOOD Prediction: ',nb_goods)    
    print('BAD Prediction: ',nb_wrongs) 
    print('Prediction good in first 4 values: ',nb_neutral) 

removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 41, 42, 47, 49, 51, 53]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1

removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 25, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 44, 45, 46, 47, 48, 49]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 14, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 35, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 36, 37, 38, 39, 40, 44, 45, 46, 47, 48, 50, 51, 52]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 46, 48]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19

removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 34, 35, 36, 37, 38, 40, 41, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 35, 36, 39, 40, 41, 42, 43, 45, 46, 47]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 16, 17, 18, 20, 21, 22, 23, 24, 25, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 

removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 4, 5, 6, 8, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 16, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 44, 45, 46, 48, 49, 50]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 22, 23, 24, 25, 26, 27, 29, 31, 32, 33, 34, 35, 38, 39, 40, 41, 42, 45, 46, 47, 49]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 42, 44, 46, 47, 48]
removeharmonics = 1 in SimulateLaue_full_np() tokeep [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28,